## Solving supply-demand problem using ACO

In neighborhood $Y$ of city $X$, there are 8 schools that collectively possess 100 microscopes for use in biology classes. These resources, however, are not uniformly distributed amongst the schools. With recent changes in student enrollment, 4 schools have more microscopes than needed while the other 4 schools are in need of additional ones. To address this issue, Dr. Rachel Carson, who is in charge of the biology department at City $X$'s School Board, decides to use a mathematical model. She chooses to use the Transportation Problem model, a strategy aimed at efficiently allocating resources while minimizing transportation costs [1]. The model represents supply $n$ and demand $m$ as unit weights of decision variables at various points in a network, with the cost of transporting a unit from a supply point to a demand point equivalent to the time or distance between nodes. This data is captured in an $n \times m$ cost matrix.

The formal statement of this integer linear programming problem is as follows [2]:

Minimize $\sum_{i \in I}\sum_{j \in J} c_{ij}x_{ij}$

Subject to the following constraints:

$\sum_{j \in J} x_{ij}\leq S_i  \quad \quad \forall i \in I$

$\sum_{i \in I} x_{ij}\geq D_j  \quad \quad \forall j \in J$

$x_{ij}\geq 0 \quad \quad \forall i \in I \quad \quad \forall j \in J$

where
* $i$ is each potential origin node
* $I$ is the complete set of potential origin nodes
* $j$ is each potential destination node
* $J$ is the complete set of potential nodes
* $x_{ij}$ is the amount to be shipped from $i \in I$ to $j \in J$
* $c_{ij}$ is per unit shipping costs between all $i, j$ pairs. Assume that this cost is 50*distance between between $i$ and $j$.
* $S_i$ is node i supply for $i \in I$
* $D_j$ is node i demand for $j \in J$

<b> References</b>

[1] Lovelace, R. Open source tools for geographic analysis in transport planning. Journal of Geographical Systems (2021). 

[2] Daskin, M. (2013) Network and Discrete Location: Models, Algorithms, and Applications. New York: John Wiley & Sons, Inc.

### Install Mixed Integer Distributed Ant Colony Optimization (<a href="http://www.midaco-solver.com/">MIDACO</a>)

Step 1	Download MIDACO python gateway <a href="http://www.midaco-solver.com/index.php/download/python">midaco.py</a> and remove .txt extension

Step 2	Download appropriate library file <a href="http://www.midaco-solver.com/index.php/download/python">midacopy.dll</a> or midacopy.so

Step 3	Store all files in the same folder (e.g. Desktop) on your PC

### Import used libraries

In [1]:
import numpy as np
import random
import midaco
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium

### Set the supply-demand problem data

<b>Impotant Note</b>: MIDACO will be used to solve a reduced problem set. MIDACO is a licensed software, it is redistributed with a limited license that enable optimizations with up to four variables (2 supply schools and 2 demand schools). For higher number of school, you need to obtain an unlimited license that support up to 100,000 variables. Please refer to the specific <a href="http://www.midaco-solver.com/index.php/more/price">commercial or academic agreement</a> made with the code developers.

In [2]:
supply_schools = [1, 3] # schools with microscopes available
demand_schools = [2, 4] # schools with microscopes requested
amount_supply = [20, 30] # number of microscopes available at each school
amount_demand = [5, 45] # number of microscopes requested at each school
n_var=len(supply_schools)*len(demand_schools)
n_constr=len(supply_schools)

### Visualize the problem data

In [3]:
geolocator = Nominatim(user_agent="SupplyDemand") # create geolocator object
location = geolocator.geocode("Toronto, Ontario") # get coordinates of Toronto

In [4]:
# Function to generate random locations around a center point
def generate_random(number, center_point):
    lat, lon = center_point
    coords = [(random.uniform(lat - 0.01, lat + 0.01), random.uniform(lon - 0.01, lon + 0.01)) for _ in range(number)]
    return coords

np.random.seed(0)  # for reproducibility
supply_coords = generate_random(len(supply_schools), [location.latitude, location.longitude])
demand_coords = generate_random(len(demand_schools), [location.latitude, location.longitude])

In [5]:
# Create a map centered at Downtown Toronto
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=15, scrollWheelZoom=False, dragging=False)

# Add markers for supply schools
for i, coord in zip(supply_schools, supply_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="home", color='red'), popup=f'Supply School {i+1}').add_to(m)

# Add markers for demand schools
for i, coord in zip(demand_schools, demand_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="flag", color='blue'), popup=f'Demand School {i+1}').add_to(m)

# Show the map
m

### Define Cost matrix

In [6]:
distances = []
for supply in supply_coords:
    row = []
    for demand in demand_coords:
        row.append(geodesic(supply, demand).kilometers)
    distances.append(row)

distances = np.array(distances)

cost_matrix=50*distances

### Define the problem

In [7]:
def problem_function(x):

    f = [0.0]*1 # Initialize array for objectives F(X)
    g = [0.0]*n_constr # Initialize array for constraints G(X)

    # Objective functions F(X)  
    f[0] = np.sum(np.multiply(cost_matrix.flatten(), x))  # Objective function

    soln=np.reshape(x, (len(supply_schools), len(demand_schools)))  
   
    # # # Inequality constraints G(X) >= 0 MUST COME SECOND in g[me:m-1] 
    # for i in range(n_constr):
    #     total_supply = np.sum(soln[i])
    #     total_demand = np.sum(soln.T[i])

    total_supply = soln.sum(axis=1)
    total_demand = soln.sum(axis=0)        

    g[0] = (np.all(total_supply>=amount_supply) and np.all(total_demand>=amount_demand))-1        

    return f,g

### Step 1: Problem definition

In [8]:
key = b'MIDACO_LIMITED_VERSION___[CREATIVE_COMMONS_BY-NC-ND_LICENSE]'
problem = {} # Initialize dictionary containing problem specifications
option  = {} # Initialize dictionary containing MIDACO options

problem['@'] = problem_function # Handle for problem function name

# STEP 1.A: Problem dimensions
problem['o']  = 1  # Number of objectives 
problem['n']  = n_var  # Number of variables (in total) 
problem['ni'] = n_var  # Number of integer variables (0 <= ni <= n) 
problem['m']  = n_constr  # Number of constraints (in total) 
problem['me'] = 0  # Number of equality constraints (0 <= me <= m) 

# STEP 1.B: Lower and upper bounds 'xl' & 'xu'  
problem['xl'] = [0.0]*n_var
problem['xu'] = [30.0]*n_var

# STEP 1.C: Starting point 'x'  
problem['x'] = problem['xl'] # Here for example: starting point = lower bounds

### Step 2: Choose stopping criteria and printing options

In [9]:
# STEP 2.A: Stopping criteria 
option['maxeval'] = 10000     # Maximum number of function evaluation (e.g. 1000000) 
option['maxtime'] = 60*60*24  # Maximum time limit in Seconds (e.g. 1 Day = 60*60*24) 

# STEP 2.B: Printing options  
option['printeval'] = 1000  # Print-Frequency for current best solution (e.g. 1000) 
option['save2file'] = 1     # Save SCREEN and SOLUTION to TXT-files [0=NO/1=YES]

### Step 3: Choose MIDACO parameters (FOR ADVANCED USERS)

In [10]:
option['param1']  = 0.0  # ACCURACY  
option['param2']  = 0.0  # SEED  
option['param3']  = 0.0  # FSTOP  
option['param4']  = 0.0  # ALGOSTOP  
option['param5']  = 0.0  # EVALSTOP  
option['param6']  = 0.0  # FOCUS  
option['param7']  = 0.0  # ANTS  
option['param8']  = 0.0  # KERNEL  
option['param9']  = 0.0  # ORACLE  
option['param10'] = 0.0  # PARETOMAX
option['param11'] = 0.0  # EPSILON  
option['param12'] = 0.0  # BALANCE
option['param13'] = 0.0  # CHARACTER

### Step 4: Choose Parallelization Factor

In [11]:
option['parallel'] = 0 # Serial: 0 or 1, Parallel: 2,3,4,5,6,7,8...

### Run MIDACO and print the results

In [12]:
solution = midaco.run( problem, option, key)

print(f"Optimal solution: {np.reshape(solution['x'], (len(supply_schools), len(demand_schools)))}")
print(f"Optimal value of the objective function: {np.round(solution['f'],4)}")
print()

Optimal solution: [[ 0. 30.]
 [15. 15.]]
Optimal value of the objective function: [1919.3192]



### Print the soltuion

In [13]:
soln = solution['x']

# For each supply point
for i in range(len(amount_supply)):
    print(f"Supply School({supply_schools[i]}): {' + '.join(['soln['+str(j)+']' for j in range(i*len(supply_schools), (i+1)*len(supply_schools))])} <= {amount_supply[i]} or {' + '.join(map(str, soln[i*len(supply_schools):(i+1)*len(supply_schools)]))} <= {amount_supply[i]} or {sum(soln[i*len(supply_schools):(i+1)*len(supply_schools)])} <= {amount_supply[i]}")

# For each demand point
for j in range(len(amount_demand)):
    print(f"Demand School({demand_schools[j]}): {' + '.join(['soln['+str(i*len(amount_demand)+j)+']' for i in range(len(amount_demand))])} >= {amount_demand[j]} or {' + '.join(map(str, [soln[i*len(amount_demand)+j] for i in range(len(amount_demand))]))} >= {sum(soln[i*len(amount_demand)+j] for i in range(len(amount_demand)))} or {sum(soln[i*len(amount_demand)+j] for i in range(len(amount_demand)))} >= {amount_demand[j]}") 

# Print shipping cost
print(f"Shipping cost =  {solution['f']} $")

Supply School(1): soln[0] + soln[1] <= 20 or 0.0 + 30.0 <= 20 or 30.0 <= 20
Supply School(3): soln[2] + soln[3] <= 30 or 15.0 + 15.0 <= 30 or 30.0 <= 30
Demand School(2): soln[0] + soln[2] >= 5 or 0.0 + 15.0 >= 15.0 or 15.0 >= 5
Demand School(4): soln[1] + soln[3] >= 45 or 30.0 + 15.0 >= 45.0 or 45.0 >= 45
Shipping cost =  [1919.3192442452619] $


### Print MIDACO screen

In [14]:
file_path = "MIDACO_SCREEN.TXT"

try:
    with open(file_path, "r") as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print("File not found.")


 MIDACO 6.0 (www.midaco-solver.com)
 ----------------------------------

 LICENSE-KEY:  MIDACO_LIMITED_VERSION___[CREATIVE_COMMONS_BY-NC-ND_LICENSE]

 ----------------------------------------
 | OBJECTIVES    1 | PARALLEL         1 |
 |--------------------------------------|
 | N             4 | MAXEVAL      10000 |
 | NI            4 | MAXTIME      86400 |
 | M             2 | PRINTEVAL     1000 |
 | ME            0 | SAVE2FILE        1 |
 |--------------------------------------|
 | PARAMETER:    All by default (0)     |
 ----------------------------------------

 [     EVAL,    TIME]        OBJECTIVE FUNCTION VALUE         VIOLATION OF G(X)
 ------------------------------------------------------------------------------
 [        1,       0]        F(X):         0.00000000         VIO:     1.000000
 [     1000,       1]        F(X):      1924.93936650         VIO:     0.000000
 [     2000,       1]        F(X):      1924.93936650         VIO:     0.000000
 [     3000,       1]       

### Print the detials

In [15]:
file_path = "MIDACO_SOLUTION.TXT"

try:
    with open(file_path, "r") as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print("File not found.")

 MIDACO - SOLUTION
 -----------------

 This file saves the current best solution X found by MIDACO.
 This file is updated after every PRINTEVAL function evaluation,
 if X has been improved.



            CURRENT BEST SOLUTION
 --------------------------------------------
 EVAL:         1,  TIME:    0.00,  IFLAG:  -3 
 --------------------------------------------
 f[0] =                     0.000000000000000 
 --------------------------------------------
 VIOLATION OF G(X)             1.000000000000
 --------------------------------------------
 g[   0] =     -1.00000000  (IN-EQUAL CONSTR)  <---  INFEASIBLE  ( G < 0 )
 g[   1] =      0.00000000  (IN-EQUAL CONSTR)
 --------------------------------------------         BOUNDS-PROFIL    
 x[   0] =                 0.000000000000000;  # XL___________________ 
 x[   1] =                 0.000000000000000;  # XL___________________ 
 x[   2] =                 0.000000000000000;  # XL___________________ 
 x[   3] =                 0.0000000000

### Visualize the solution

In [16]:
# Normalize function
def normalize(lst):
    s = sum(lst)
    return list(map(lambda x: (x/s)*10, lst))

# Normalize soln array
soln_normalized = normalize(soln)

# Define a color list
colors = ['cyan', 'brown', 'orange', 'purple']

# Create a map centered at Downtown Toronto
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=15, scrollWheelZoom=False, dragging=False)

# Add markers for supply schools
for i, coord in zip(supply_schools, supply_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="home", color='red'), popup=f'Supply School {i+1}').add_to(m)

# Add markers for demand schools
for i, coord in zip(demand_schools, demand_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="flag", color='blue'), popup=f'Demand School {i+1}').add_to(m)

# Add lines (edges) between supply and demand schools
for i in range(len(supply_schools)):
    for j in range(len(demand_schools)):
        soln_value = soln[i*len(demand_schools) + j]
        folium.PolyLine(locations=[supply_coords[i], demand_coords[j]], color=colors[i % len(colors)],  weight=5*soln_normalized[i*len(demand_schools) + j], popup=folium.Popup(f'# of microscopes: {soln_value}')).add_to(m)
             
# Show the map
m